# NLP Standard Project:

- Students: **Matteo Belletti**, **Alessandro Pasi**, **Stricescu Razvan Ciprian**.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Data loading and preprocessing:

In [7]:
# open json in project_data_MELD folder
import json
with open('project_data_MELD/MELD_train_efr.json') as f:
    data = json.load(f)
print(f"Number of samples: {len(data)}")
print(f"Example of a sample: {data[0]}")

Number of samples: 4000
Example of a sample: {'episode': 'utterance_0', 'speakers': ['Chandler', 'The Interviewer', 'Chandler', 'The Interviewer', 'Chandler'], 'emotions': ['neutral', 'neutral', 'neutral', 'neutral', 'surprise'], 'utterances': ["also I was the point person on my company's transition from the KL-5 to GR-6 system.", "You must've had your hands full.", 'That I did. That I did.', "So let's talk a little bit about your duties.", 'My duties?  All right.'], 'triggers': [0.0, 0.0, 0.0, 1.0, 0.0]}


In [20]:
# Convert data to pandas dataframe
import pandas as pd
df = pd.DataFrame(data)
# drop episode and speakers columns
df = df.drop(columns=['episode', 'speakers'])
print(f"Dataframe shape: {df.shape}")
print(f"Dataframe columns: {df.columns}")

Dataframe shape: (4000, 3)
Dataframe columns: Index(['emotions', 'utterances', 'triggers'], dtype='object')


Changing nan values to zeros in order to avoid errors.

In [47]:
# Chech for NaN values on triggers column
nan_counter = 0
for row in df["triggers"]:
    for elem in row:
        if elem != 1 and elem != 0:
            nan_counter += 1

print(f"Number of NaN values changed: {nan_counter}")

Number of NaN values changed: 9


In [48]:
df["triggers"] = df["triggers"].apply(lambda x: [0 if elem != 1 and elem != 0 else elem for elem in x])